In [63]:
#Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly

In [60]:
#Create dataframes for all the files
artist_attributes_df = pd.read_csv('/content/artist_attributes.csv')
artist_to_imdb_df = pd.read_csv('/content/artist_to_imdb.csv')
imdb_attributes_df = pd.read_csv('/content/imdb_attributes.csv')
imdb_genres_df = pd.read_csv('/content/imdb_genres.csv')
links_df = pd.read_csv('/content/links.csv')
movie_genres_df = pd.read_csv('/content/movie_genres.csv')
movie_title_keywords_df = pd.read_csv('/content/movie_title_keywords.csv')
movies_df = pd.read_csv('/content/movies.csv')
ratings_df = pd.read_csv('/content/ratings.csv')
tmdb_attributes_df = pd.read_csv('/content/tmdb_attributes.csv')
tmdb_keywords_df = pd.read_csv('/content/tmdb_keywords.csv')
tmdb_overview_keywords_df = pd.read_csv('/content/tmdb_overview_keywords.csv')

In [62]:
#Check total num of rows for each dataframe
df_names = [artist_attributes_df, artist_to_imdb_df, imdb_attributes_df, imdb_genres_df, links_df, movie_genres_df, movie_title_keywords_df, movies_df, ratings_df, tmdb_attributes_df, tmdb_keywords_df, tmdb_overview_keywords_df]
names = ['artist_attributes_df', 'artist_to_imdb_df', 'imdb_attributes_df', 'imdb_genres_df', 'links_df', 'movie_genres_df', 'movie_title_keywords_df', 'movies_df', 'ratings_df', 'tmdb_attributes_df', 'tmdb_keywords_df', 'tmdb_overview_keywords_df']

for i in range(0, len(df_names)):
  print(f'{names[i]} : {len(df_names[i])} rows')

artist_attributes_df : 175719 rows
artist_to_imdb_df : 627212 rows
imdb_attributes_df : 14761 rows
imdb_genres_df : 33040 rows
links_df : 9742 rows
movie_genres_df : 22084 rows
movie_title_keywords_df : 47663 rows
movies_df : 27278 rows
ratings_df : 1324194 rows
tmdb_attributes_df : 4803 rows
tmdb_keywords_df : 36194 rows
tmdb_overview_keywords_df : 50258 rows


##Clean movie genres

In [65]:
movie_genres_df.head()

,Unnamed: 0,genre,movieID,title
0,0,Adventure,1,Toy Story (1995)
1,1,Animation,1,Toy Story (1995)
2,2,Children,1,Toy Story (1995)
3,3,Comedy,1,Toy Story (1995)
4,4,Fantasy,1,Toy Story (1995)


In [66]:
#Remove Unnecessary column
movie_genres_df = movie_genres_df.drop('Unnamed: 0', axis=1)

In [67]:
#group data by its movieID to consolidate all the genres into a list under a new column called 'genre_list'
movie_genres_group_df = movie_genres_df.groupby(['movieID'])['genre'].apply(list).rename('genre_list').reset_index()

In [69]:
#Create a separate dataframe that only contains rows where genre is equal only to Children
movie_genres_children_df = movie_genres_df[movie_genres_df['genre'] == 'Children'].reset_index(drop=True).drop('genre', axis=1)

In [71]:
#Join two dataframes together to get the final result for children genre movies
movie_genres_children_df = movie_genres_children_df.merge(movie_genres_group_df, how='inner', on='movieID')

In [72]:
movie_genres_children_df

,movieID,title,genre_list
0,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]"
1,2,Jumanji (1995),"[Adventure, Children, Fantasy]"
2,8,Tom and Huck (1995),"[Adventure, Children]"
3,13,Balto (1995),"[Adventure, Animation, Children]"
4,27,Now and Then (1995),"[Children, Drama]"
...,...,...,...
659,182731,Pixel Perfect (2004),"[Children, Comedy, SciFi]"
660,183301,The Tale of the Bunny Picnic (1986),[Children]
661,184987,A Wrinkle in Time (2018),"[Adventure, Children, Fantasy, SciFi]"
662,187541,Incredibles 2 (2018),"[Action, Adventure, Animation, Children]"
